In [28]:
import pandas as pd
import gcsfs

In [29]:
url = 'gs://taxi_data_analytics-gcp/yellow_taxi_2023_05_01.parquet'
df = pd.read_parquet(url)

In [30]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-05-01 00:33:13,2023-05-01 00:53:01,0.0,7.80,1.0,N,138,43,1,33.80,7.75,0.5,8.60,0.00,1.0,51.65,0.0,1.75
1,1,2023-05-01 00:42:49,2023-05-01 01:11:18,2.0,8.10,1.0,N,138,262,1,35.90,10.25,0.5,9.50,0.00,1.0,57.15,2.5,1.75
2,1,2023-05-01 00:56:34,2023-05-01 01:13:39,2.0,9.10,1.0,N,138,141,1,35.20,10.25,0.5,10.70,6.55,1.0,64.20,2.5,1.75
3,2,2023-05-01 00:00:52,2023-05-01 00:20:12,1.0,8.21,1.0,N,138,140,1,33.10,6.00,0.5,2.24,0.00,1.0,47.09,2.5,1.75
4,1,2023-05-01 00:05:50,2023-05-01 00:19:41,0.0,7.90,1.0,N,138,263,1,31.00,10.25,0.5,9.85,6.55,1.0,59.15,2.5,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3414227,2,2023-05-01 23:26:20,2023-05-01 23:28:54,NaN,0.54,NaN,None,234,170,0,7.11,0.00,0.5,0.00,0.00,1.0,11.11,NaN,NaN
3414228,2,2023-05-01 23:38:00,2023-05-01 23:49:00,NaN,1.39,NaN,None,230,233,0,10.51,0.00,0.5,2.90,0.00,1.0,17.41,NaN,NaN
3414229,2,2023-05-01 23:32:18,2023-05-01 23:48:38,NaN,3.68,NaN,None,263,50,0,17.38,0.00,0.5,3.21,0.00,1.0,24.59,NaN,NaN
3414232,2,2023-05-01 23:26:01,2023-05-01 23:53:34,NaN,6.47,NaN,None,79,89,0,32.72,0.00,0.5,0.00,0.00,1.0,36.72,NaN,NaN


In [31]:
df = df.drop_duplicates().reset_index(drop=True)

In [32]:
df['trip_id'] = df.index

In [33]:
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour

In [34]:
datetime_dim['pick_date'] = datetime_dim['tpep_pickup_datetime'].dt.day

In [35]:
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month

In [36]:
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour

In [37]:
datetime_dim['drop_date'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday
datetime_dim['datetime_id'] = datetime_dim.index

In [38]:
datetime_dim = datetime_dim[['datetime_id','tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pick_hour',
       'pick_date', 'pick_month', 'pick_year', 'pick_weekday', 'drop_hour',
       'drop_date', 'drop_year', 'drop_weekday']]

In [39]:
passenger_count_dim = df[['passenger_count']].reset_index(drop = True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]

In [40]:
trip_distance_dim = df[['trip_distance']].reset_index(drop = True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]

In [41]:
ratecode = {1:'Standard rate',
2:'JFK',
3:'Newark',
4:'Nassau or Westchester',
5:'Negotiated fare',
6:'Group ride'}
rate_code_dim = df[['RatecodeID']].reset_index(drop = True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['ratecode_name'] = rate_code_dim['RatecodeID'].map(ratecode)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','ratecode_name']]

In [42]:
paymenttype = {1:'Credit card',
2:'Cash',
3:'No charge',
4:'Dispute',
5:'Unknown',
6:'Voided trip'}
payment_type_dim = df[['payment_type']].reset_index(drop = True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(paymenttype)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]

In [43]:
pickup_location_dim = df[['PULocationID']].reset_index(drop = True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[['pickup_location_id','PULocationID']]

In [44]:
dropoff_location_dim = df[['DOLocationID']].reset_index(drop = True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id','DOLocationID']]

In [45]:
fact_table = df.merge(passenger_count_dim, left_on = 'trip_id', right_on = 'passenger_count_id').\
merge(pickup_location_dim, left_on = 'trip_id', right_on = 'pickup_location_id').\
merge(dropoff_location_dim, left_on = 'trip_id', right_on = 'dropoff_location_id').\
merge(rate_code_dim, left_on = 'trip_id', right_on = 'rate_code_id').\
merge(datetime_dim, left_on = 'trip_id', right_on = 'datetime_id').\
merge(trip_distance_dim, left_on = 'trip_id', right_on = 'trip_distance_id').\
merge(payment_type_dim, left_on = 'trip_id', right_on = 'payment_type_id')


In [46]:
fact_table = fact_table[['trip_id','VendorID','datetime_id','passenger_count_id','trip_distance_id','pickup_location_id','dropoff_location_id','rate_code_id','payment_type_id','store_and_fwd_flag','fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge','total_amount','congestion_surcharge','Airport_fee']]

In [53]:
fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,pickup_location_id,dropoff_location_id,rate_code_id,payment_type_id,store_and_fwd_flag,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,0,1,0,0,0,0,0,0,0,N,33.80,7.75,0.5,8.60,0.00,1.0,51.65,0.0,1.75
1,1,1,1,1,1,1,1,1,1,N,35.90,10.25,0.5,9.50,0.00,1.0,57.15,2.5,1.75
2,2,1,2,2,2,2,2,2,2,N,35.20,10.25,0.5,10.70,6.55,1.0,64.20,2.5,1.75
3,3,2,3,3,3,3,3,3,3,N,33.10,6.00,0.5,2.24,0.00,1.0,47.09,2.5,1.75
4,4,1,4,4,4,4,4,4,4,N,31.00,10.25,0.5,9.85,6.55,1.0,59.15,2.5,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98877,98877,2,98877,98877,98877,98877,98877,98877,98877,None,7.11,0.00,0.5,0.00,0.00,1.0,11.11,NaN,NaN
98878,98878,2,98878,98878,98878,98878,98878,98878,98878,None,10.51,0.00,0.5,2.90,0.00,1.0,17.41,NaN,NaN
98879,98879,2,98879,98879,98879,98879,98879,98879,98879,None,17.38,0.00,0.5,3.21,0.00,1.0,24.59,NaN,NaN
98880,98880,2,98880,98880,98880,98880,98880,98880,98880,None,32.72,0.00,0.5,0.00,0.00,1.0,36.72,NaN,NaN


In [55]:
datetime_dim.describe()

,datetime_id,tpep_pickup_datetime,tpep_dropoff_datetime,pick_hour,pick_date,pick_month,pick_year,pick_weekday,drop_hour,drop_date,drop_year,drop_weekday
count,98882.000000,98882,98882,98882.000000,98882.000000,98882.000000,98882.000000,98882.000000,98882.000000,98882.000000,98882.000000,98882.000000
mean,49440.500000,2023-05-01 07:16:20.832496896,2023-05-01 07:33:17.219959296,14.190469,1.003256,5.000051,2022.999140,0.000587,14.464847,1.001203,2022.999151,0.000243
min,0.000000,2002-12-31 23:28:55,2002-12-31 23:33:07,0.000000,1.000000,1.000000,2002.000000,0.000000,0.000000,1.000000,2002.000000,0.000000
25%,24720.250000,2023-05-01 10:56:53.249999872,2023-05-01 11:14:44.249999872,10.000000,1.000000,5.000000,2023.000000,0.000000,11.000000,1.000000,2023.000000,0.000000
50%,49440.500000,2023-05-01 15:10:18,2023-05-01 15:29:56,15.000000,1.000000,5.000000,2023.000000,0.000000,15.000000,1.000000,2023.000000,0.000000
75%,74160.750000,2023-05-01 18:48:10.750000128,2023-05-01 19:04:18.750000128,18.000000,1.000000,5.000000,2023.000000,0.000000,19.000000,1.000000,2023.000000,0.000000
max,98881.000000,2023-05-01 23:56:18,2023-05-01 23:59:54,23.000000,31.000000,12.000000,2023.000000,6.000000,23.000000,31.000000,2023.000000,6.000000
std,28544.918996,NaN,NaN,5.320925,0.308766,0.043487,0.122628,0.055989,5.312477,0.187677,0.121426,0.032740
